In [58]:
import subprocess
import json
import os
from constants import BTC, BTCTEST, ETH
from pprint import pprint
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware

In [59]:
# connect Web3
w3 = Web3(Web3.HTTPProvider(os.getenv('WEB3_PROVIDER', 'http://127.0.0.1:8545')))

In [60]:
# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [61]:
# include mnemonic with prefunded test tokens 
mnemonic = os.getenv("MNEMONIC", "become shaft talk pave front auction employ share toe valve miracle rain")

In [77]:
mnemonic

'become shaft talk pave front auction employ share toe valve miracle rain'

In [ ]:
command = !php hd-wallet-derive.php -g --mnemonic="suffer elephant glance leopard point device solar whip tip test banner inhale" --cols=path,address,privkey,pubkey --format=json

In [99]:
command[1]

'[{"path":"m\\/44\'\\/0\'\\/0\'\\/0\\/0","address":"1F3FCkEjGiGdw3YtRySWDu7EV1MvYjx4r3","privkey":"L3Kk89GAGGK5HU5sLCtAVABzxTsccFL9kYcVzwxozuAAbumpwzEE","pubkey":"02c1d33cedb7aa55e9679c1bb03364b21b4d5e2a35bad499859aa3847eae1a158f"},{"path":"m\\/44\'\\/0\'\\/0\'\\/0\\/1","address":"1KQbUN91kJS8KhJkWRMrzxRQdckhNQ6MmT","privkey":"L4nLqcrz1ZTzouy9QbwZczmpzLrGqhii4QqiAEcHtK4MD9SorMhx","pubkey":"0397f61c19c4bdc1fef0b82b2728e204b8002578b5e771337fe27e5e16c5beb720"},{"path":"m\\/44\'\\/0\'\\/0\'\\/0\\/2","address":"1Pu4PQE2Nrkmk8WUAMnST9L4faF5Nvji4t","privkey":"L4GjazZhfYagYKWgGycpfg7afUmhPXu7yRnqEz7YXXfvtDyUftVo","pubkey":"0390372063a93a99457d38a310fb3c087e56837bbf91fd1076f7755c90c494c700"},{"path":"m\\/44\'\\/0\'\\/0\'\\/0\\/3","address":"13L1eqvA22A2sy4JYwkXwvsRYUct4m33sU","privkey":"Kz1mFLFCeK8LqKmhJUVptzU62JoQKrQrNMxXUjPq43yjMZ1X7eyR","pubkey":"030e9fa3ccf109df47964d481d0f6a102cca573ee3567a7ccaf473c0f0f2739443"},{"path":"m\\/44\'\\/0\'\\/0\'\\/0\\/4","address":"1K83AaxASmEJ56ym2QxAgtGTZYud

In [75]:
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f"php ./derive -g --mnemonic='{mnemonic}' --coin={coin} --numderive={depth} --cols=path,address,privkey,pubkey --format=json"
    p = subprocess.Popen(command,
        stdout=subprocess.PIPE,
        shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [64]:
command = './derive -g --mnemonic="INSERT HERE" --cols=path,address,privkey,pubkey --format=json'

p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
output, err = p.communicate()
p_status = p.wait()

keys = json.loads(output)
print(keys)

In [87]:
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f"php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic='{mnemonic}' --cols=path,address,privkey,pubkey --coin={coin} --numderive={depth} --format=json"
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    
    output, err = p.communicate()
    p_status = p.wait()
    print(output)
    return json.loads(output)

In [67]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [68]:
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 wei
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account.address, "amount": value })
        return {
            "to": to,
            "from": account.address,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account.address),
            "chainId": w3.net.chainId
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

In [69]:
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.signTransaction(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

In [ ]:
coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

In [ ]:
pprint(coins)